INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-TED-Bert-en-vi"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/Script/TED-20210226-2049


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-02-26 20:49:32--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-02-26 20:49:34--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  48,3KB/s    in 3m 15s  

2021-02-26 20:52:50 (45,3 KB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/904410

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x 3 chinh chinh     4096 Thg 2  26 20:53 .
drwxrwxr-x 4 chinh chinh     4096 Thg 2  26 20:52 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 2  26 20:52 data_bin
-rw-rw-r-- 1 chinh chinh  1260181 Thg 2  26 20:53 en_test
-rw-rw-r-- 1 chinh chinh 10011509 Thg 2  26 20:53 en_train
-rw-rw-r-- 1 chinh chinh  1249342 Thg 2  26 20:53 en_valid
-rw-rw-r-- 1 chinh chinh  9044100 Thg 2  26 20:52 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 chinh chinh  1624771 Thg 2  26 20:53 vi_test
-rw-rw-r-- 1 chinh chinh 12896000 Thg 2  26 20:53 vi_train
-rw-rw-r-- 1 chinh chinh  1609835 Thg 2  26 20:53 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# # model = SentenceTransformer('LaBSE')
# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("This is a red cat with a hat.")
# emb2 = model.encode("Have you seen my red cat?")

# cos_sim = util.pytorch_cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)

In [8]:
# print(cos_sim.numpy()[0][0])

In [35]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  273.74039673805237


Test Cosin similarity

In [34]:
import faiss

start = time.time()
index = faiss.IndexFlatL2(768)
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  44.06273579597473
96764
96764
[[    0 39423 35305 72079 32103 85735]
 [    1 81481 50721 15775  1179 11991]
 [    2   250 72009 40466 66315 93518]
 ...
 [96761 50580 14876 19269 69832 50735]
 [96762 81748 24188 45756 69367  3000]
 [96763  8985 62485   200 21184 58797]]
[[0.0000000e+00 7.7359259e-01 7.9088068e-01 8.7716556e-01 8.8684976e-01
  9.2159152e-01]
 [0.0000000e+00 6.8074703e-01 6.8469703e-01 7.2336924e-01 7.3071086e-01
  7.4148726e-01]
 [1.1920929e-07 9.4939709e-01 9.5867610e-01 9.6002400e-01 9.7436512e-01
  9.7728646e-01]
 ...
 [2.3841858e-07 7.6802468e-01 7.9690385e-01 8.3492100e-01 8.4242499e-01
  8.6041594e-01]
 [3.5762787e-07 9.7901368e-01 1.0931816e+00 1.1214151e+00 1.1353327e+00
  1.1470406e+00]
 [0.0000000e+00 8.6880016e-01 9.6778297e-01 9.9761200e-01 9.9859607e-01
  9.9874771e-01]]


In [31]:

i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

59075
0.8596766
0.85967684


In [32]:
start = time.time()
saperate = " || "
threshold = 0.8
src_label = "S"
emb_label = "E"

out_file_write = open("en_train_EM", 'w')
out_file_write_score = open("en_train_EM_score", 'w')
out_file_write_factor = open("en_train_EM_factor", 'w')

for i in range(len(lst_srcfile)):
    score = distance[i][1]
    content = ""
    content_label_arr = []
    if score >= threshold:
        best_simi_index = index[i][1]
        content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()
        
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label.append(emb_label)
        content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())
        
        content_label_arr = " ".join(content_label)
        
        
   
    else:
        content = lst_srcfile[i].strip()
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label_arr = " ".join(content_label)
        
        
    if i<10:
        print(content)
        print(len(content.split()))  
        print(len(content_label))
        print("======================")
        

    out_file_write.writelines(content+ "\n")    
    out_file_write_factor.writelines(content_label_arr + "\n")
    out_file_write_score.writelines(str(score) + "\n")
        

out_file_write.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

It &apos;s like the guy is sitting next to you , whispering who knows what into your ear .
19
19
They go bad very quickly if you don &apos;t , and if you don &apos;t have stable power grid , this doesn &apos;t happen , so kids die .
29
29
Now let me ask you this question : This past week at TED , how many of you , when you saw vulnerability up here , thought it was pure courage ? || Khi tôi trở thành &quot; Vulnerability TED &quot; , như một nhân vật hành động -- kiểu như Ninja Barbie , nhưng tôi là &quot; Vulnerability TED &quot; -- Tôi nghĩ rằng tôi sẽ để lại những xấu hổ đó đằng sau , vì tôi đã trải qua sáu năm nghiên cứu về sự xấu hổ trước khi tôi thực sự bắt đầu viết và nói về sự dễ bị tổn thương .
106
106
My archive : 175 films , my 16-millimeter negative , all my books , my dad &apos;s books , my photographs . || Chúng tôi đăng ký mua rất nhiều tạp chí , nghiên cứu rất nhiều , nhưng thật sự , khi bạn nhìn xung quanh , ở đâu cũng thấy những hình ảnh như thế này
58
58
Now , a few 